# Lesson 8: Exercises

## Load CADETSettings

This imports all the standard libraries and provides some helper functions.

In [ ]:
%run ../utils.ipynb

## Model templates

In [ ]:
def create_reference_model():
    reference_model = create_dextran_model()
    
    reference_model.root.input.model.unit_001.adsorption_model = 'MULTI_COMPONENT_LANGMUIR'

    reference_model.root.input.model.unit_001.adsorption.is_kinetic = 1
    reference_model.root.input.model.unit_001.adsorption.mcl_ka = [1.144,]
    reference_model.root.input.model.unit_001.adsorption.mcl_kd = [2.0e-3,]
    reference_model.root.input.model.unit_001.adsorption.mcl_qmax = [4.88,]

    reference_model.root.input.model.unit_001.film_diffusion = [2.0E-7,]
    reference_model.root.input.model.unit_001.par_diffusion = [1e-8,]
    reference_model.root.input.model.unit_001.discretization.nbound = [1,]

    #solver settings
    reference_model.root.input.solver.sections.section_times = [0.0, 3000.0, 9500.0]

    #set the times that the simulator writes out data for
    reference_model.root.input.solver.user_solution_times = np.linspace(0, 9500, 9501)

    run_simulation(reference_model, 'langmuir_reference.h5')

## Exercise 1: 

Read experimental data from the `langmuir_experiment.csv` and try to determine the values for `mcl_ka`, `mcl_kd`, and `mcl_qmax`.

***Task:*** Compare the gradient approach with different starting values `match.seeds = [(ka, kd, qmax), ]`. 
Also try setting up the `NSGA3` optimizer.

***Hint:*** The `create_reference_model()` function also creates the necessary `.h5` file.

In [ ]:
create_reference_model()

In [ ]:
base_dir = Path('./').absolute()
match_config = Dict()
match_config.CADETPath = Cadet.cadet_path
match_config.baseDir = base_dir.as_posix()
match_config.resultsDir = 'results_exercise'

parameter1 = Dict()
parameter1.transform = 'auto'
parameter1.component = 0
parameter1.bound = 0
parameter1.location = '/input/model/unit_001/adsorption/MCL_KA'
parameter1.min = 1e-5
parameter1.max = 1e5

parameter2 = Dict()
parameter2.transform = 'auto'
parameter2.component = 0
parameter2.bound = 0
parameter2.location = '/input/model/unit_001/adsorption/MCL_KD'
parameter2.min = 1e-5
parameter2.max = 1e5

parameter3 = Dict()
parameter3.transform = 'auto'
parameter3.component = 0
parameter3.bound = 0
parameter3.location = '/input/model/unit_001/adsorption/MCL_QMAX'
parameter3.min = 1
parameter3.max = 20

match_config.parameters = [parameter1, parameter2, parameter3]

experiment1 = Dict()
experiment1.csv = 'resources/langmuir_experiment.csv'
experiment1.isotherm = '/output/solution/unit_002/SOLUTION_OUTLET_COMP_000'
experiment1.HDF5 = 'langmuir_reference.h5'
experiment1.name = 'main'

feature1 = Dict()
feature1.name = 'Pulse'
feature1.type = 'SSE'

experiment1.features = [feature1,]

match_config.experiments = [experiment1,]

match_config.searchMethod = 'NSGA3'
match_config.population = 12
match_config.stallGenerations = 10
match_config.finalGradRefinement = True
match_config.gradVector = True

In [ ]:
from CADETMatch.jupyter import Match

match_file = base_dir / 'langmuir.json'

with open(match_file, 'w') as json_file:
    json.dump(match_config.to_dict(), json_file, indent='\t')

match = Match(match_file)
match.start_sim()

In [ ]:
match.plot_corner()

In [ ]:
match.plot_best()

## Exercise 2: auto_keq 

For the same example, make use of the `auto_keq` parameter transform feature of `CADET-Match`.

***Hint:*** For this feature to work, you need to provide a list of locations for the corresponding `ka` and `kd` value of the selected isotherm.

In [ ]:
base_dir = Path('./').absolute()
match_config = Dict()
match_config.CADETPath = Cadet.cadet_path
match_config.baseDir = base_dir.as_posix()
match_config.resultsDir = 'results_exercise_keq'

parameter1 = Dict()
parameter1.transform = 'auto_keq'
parameter1.component = 0
parameter1.bound = 0
parameter1.location = ['/input/model/unit_001/adsorption/MCL_KA', '/input/model/unit_001/adsorption/MCL_KD']
parameter1.minKA = 1e-5
parameter1.maxKA = 1e5
parameter1.minKEQ = 1e-4
parameter1.maxKEQ = 1e4

parameter2 = Dict()
parameter2.transform = 'auto'
parameter2.component = 0
parameter2.bound = 0
parameter2.location = '/input/model/unit_001/adsorption/MCL_QMAX'
parameter2.min = 1
parameter2.max = 20

match_config.parameters = [parameter1, parameter2]

experiment1 = Dict()
experiment1.csv = 'resources/langmuir_experiment.csv'
experiment1.isotherm = '/output/solution/unit_002/SOLUTION_OUTLET_COMP_000'
experiment1.HDF5 = 'langmuir_reference.h5'
experiment1.name = 'main'

feature1 = Dict()
feature1.name = 'Pulse'
feature1.type = 'SSE'

experiment1.features = [feature1,]

match_config.experiments = [experiment1,]


match_config.searchMethod = 'NSGA3'
match_config.population = 12
match_config.stallGenerations = 10
match_config.finalGradRefinement = True
match_config.gradVector = True

In [ ]:
from CADETMatch.jupyter import Match

match_file = base_dir / 'langmuir_keq.json'

with open(match_file, 'w') as json_file:
    json.dump(match_config.to_dict(), json_file, indent='\t')

match = Match(match_file)
match.start_sim()

In [ ]:
match.plot_corner()

In [ ]:
match.plot_best()

## Exercise 3: Shape

Adding to the previous example, now use the `shape` feature type for the pulse instead of `SSE`.

In [ ]:
base_dir = Path('./').absolute()
match_config = Dict()
match_config.CADETPath = Cadet.cadet_path
match_config.baseDir = base_dir.as_posix()
match_config.resultsDir = 'results_exercise_keq_shape'

parameter1 = Dict()
parameter1.transform = 'auto_keq'
parameter1.component = 0
parameter1.bound = 0
parameter1.location = ['/input/model/unit_001/adsorption/MCL_KA', '/input/model/unit_001/adsorption/MCL_KD']
parameter1.minKA = 1e-5
parameter1.maxKA = 1e5
parameter1.minKEQ = 1e-4
parameter1.maxKEQ = 1e4

parameter2 = Dict()
parameter2.transform = 'auto'
parameter2.component = 0
parameter2.bound = 0
parameter2.location = '/input/model/unit_001/adsorption/MCL_QMAX'
parameter2.min = 1
parameter2.max = 20

match_config.parameters = [parameter1, parameter2]

experiment1 = Dict()
experiment1.csv = 'resources/langmuir_experiment.csv'
experiment1.isotherm = '/output/solution/unit_002/SOLUTION_OUTLET_COMP_000'
experiment1.HDF5 = 'langmuir_reference.h5'
experiment1.name = 'main'

feature1 = Dict()
feature1.name = 'Pulse'
feature1.type = 'Shape'

experiment1.features = [feature1,]

match_config.experiments = [experiment1,]

match_config.searchMethod = 'NSGA3'
match_config.population = 12
match_config.stallGenerations = 10
match_config.finalGradRefinement = True
match_config.gradVector = True

In [ ]:
from CADETMatch.jupyter import Match

match_file = base_dir / 'langmuir_keq_shape.json'

with open(match_file, 'w') as json_file:
    json.dump(match_config.to_dict(), json_file, indent='\t')

match = Match(match_file)
match.start_sim()

In [ ]:
match.plot_corner()

In [ ]:
match.plot_best()